In [1]:
import numpy as np
import matplotlib.pyplot as plt

import json
import tempfile
import pyspark.sql.functions as F
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType, StructType, StructField, StringType, ArrayType
from pyproj import Transformer

# Download Data

In [2]:
import os
import gdown

folder = "data"
os.makedirs(folder, exist_ok=True)

output = os.path.join(folder, "cleaned_df.csv")
url = "https://drive.google.com/uc?id=1NnlFIt5zL8i6vO5BuxQHhPTGprGn0Rbo"

if not os.path.exists(output):
    print("File not found. Downloading...")
    gdown.download(url, output, quiet=False)
    print("Download completed!")
else:
    print("File already exists. Skip downloading.")

File already exists. Skip downloading.


In [3]:
import pandas as pd

cleaned_df = pd.read_csv(output)
cleaned_df

,comment,coords,district,timestamp,count_reopen,count_reopen_log
0,บริเวณนราธิวาส แยกถนนจันทน์ ใกล้สวนสาธารณะช่อ...,"100.53764,13.70716",สาทร,2022-01-02 10:53:25.580723+00:00,0,0.0
1,มอเตอร์ไซด์จอดบนทางเท้าเป็นประจำ ฝั่งตรงข้ามตล...,"100.52674,13.70950",สาทร,2022-01-14 01:17:23.873811+00:00,0,0.0
2,มอเตอร์ไซด์จอดบนถนน ลดช่องจราจรทำให้การจราจรชะ...,"100.52678,13.70967",สาทร,2022-01-14 01:18:57.194155+00:00,0,0.0
3,ทางเท้าช่วง จันทน์ 18/5 สภาพโอเค แต่ฝาท่อเก่าแ...,"100.53025,13.70566",สาทร,2022-01-14 01:32:03.715912+00:00,0,0.0
4,อยากให้สวนหลวง ร9 และ สวนสาธารณะอื่นๆ ใน กทม เ...,"100.65816,13.68814",ประเวศ,2022-01-15 12:52:39.805944+00:00,0,0.0
...,...,...,...,...,...,...
188292,มีการตั้งเวที ร้องเพลง ส่งเสียงดัง โดยจุดเกิดเ...,"100.56189,13.73713",วัฒนา,2022-12-31 20:03:10.221437+00:00,0,0.0
188293,ซอย (ไม่ทราบชื่อ) ถนนมังกร เขตป้อมปราบศัตรูพ่า...,"100.51280,13.74480",ป้อมปราบศัตรูพ่าย,2022-12-31 20:37:37.226395+00:00,0,0.0
188294,เปิดเพลงเสียงดังตั้งแต่ตอนเช้าจนถึงตีสี่ยังไม่...,"100.42785,13.68924",ภาษีเจริญ,2022-12-31 20:52:02.074206+00:00,0,0.0
188295,เสียงดังรบกวนรำคาน นอนไม่ได้ ผู้แจ้งไม่แน่ใจว่...,"100.56181,13.73713",วัฒนา,2022-12-31 21:28:10.060293+00:00,0,0.0


# Coors Preparation

In [4]:
import geopandas as gpd
from shapely.geometry import Point

In [5]:
cleaned_df[['lng', 'lat']] = cleaned_df['coords'].str.split(',', expand=True).astype(float)
geometry = [Point(xy) for xy in zip(cleaned_df['lng'], cleaned_df['lat'])]
cleaned_df = gpd.GeoDataFrame(cleaned_df, geometry=geometry, crs="EPSG:4326")

# Drop coords
cleaned_df = cleaned_df.drop(columns=["coords"], errors='ignore')
cleaned_df

,comment,district,timestamp,count_reopen,count_reopen_log,lng,lat,geometry
0,บริเวณนราธิวาส แยกถนนจันทน์ ใกล้สวนสาธารณะช่อ...,สาทร,2022-01-02 10:53:25.580723+00:00,0,0.0,100.53764,13.70716,POINT (100.53764 13.70716)
1,มอเตอร์ไซด์จอดบนทางเท้าเป็นประจำ ฝั่งตรงข้ามตล...,สาทร,2022-01-14 01:17:23.873811+00:00,0,0.0,100.52674,13.70950,POINT (100.52674 13.7095)
2,มอเตอร์ไซด์จอดบนถนน ลดช่องจราจรทำให้การจราจรชะ...,สาทร,2022-01-14 01:18:57.194155+00:00,0,0.0,100.52678,13.70967,POINT (100.52678 13.70967)
3,ทางเท้าช่วง จันทน์ 18/5 สภาพโอเค แต่ฝาท่อเก่าแ...,สาทร,2022-01-14 01:32:03.715912+00:00,0,0.0,100.53025,13.70566,POINT (100.53025 13.70566)
4,อยากให้สวนหลวง ร9 และ สวนสาธารณะอื่นๆ ใน กทม เ...,ประเวศ,2022-01-15 12:52:39.805944+00:00,0,0.0,100.65816,13.68814,POINT (100.65816 13.68814)
...,...,...,...,...,...,...,...,...
188292,มีการตั้งเวที ร้องเพลง ส่งเสียงดัง โดยจุดเกิดเ...,วัฒนา,2022-12-31 20:03:10.221437+00:00,0,0.0,100.56189,13.73713,POINT (100.56189 13.73713)
188293,ซอย (ไม่ทราบชื่อ) ถนนมังกร เขตป้อมปราบศัตรูพ่า...,ป้อมปราบศัตรูพ่าย,2022-12-31 20:37:37.226395+00:00,0,0.0,100.51280,13.74480,POINT (100.5128 13.7448)
188294,เปิดเพลงเสียงดังตั้งแต่ตอนเช้าจนถึงตีสี่ยังไม่...,ภาษีเจริญ,2022-12-31 20:52:02.074206+00:00,0,0.0,100.42785,13.68924,POINT (100.42785 13.68924)
188295,เสียงดังรบกวนรำคาน นอนไม่ได้ ผู้แจ้งไม่แน่ใจว่...,วัฒนา,2022-12-31 21:28:10.060293+00:00,0,0.0,100.56181,13.73713,POINT (100.56181 13.73713)


# Web scraping
impact_to_public

In [6]:
from pyspark.sql import SparkSession

spark = (SparkSession.builder
        .master('local[*]')
        .appName('Spark Tutorial')
        .config('spark.ui.port', '4040')
        .getOrCreate()
        )

spark

In [7]:
import requests

In [8]:
# Department Store

url_department = "https://data.bangkok.go.th/dataset/d8f814ac-cbaf-43c3-9576-f533b2554776/resource/438101c3-5535-4fe2-bc5e-83aa73703d4a/download/department_store.csv"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
}

response = requests.get(url_department, headers=headers)
response.raise_for_status()

with tempfile.NamedTemporaryFile(delete=False, suffix=".csv") as tmp:
    tmp.write(response.content)
    temp_path = tmp.name

df_department = spark.read.csv(temp_path, header=True, inferSchema=True)

df_department.printSchema()
df_department.show(5)

root
 |-- id_depart: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- address: string (nullable = true)
 |-- tel: string (nullable = true)
 |-- time: string (nullable = true)
 |-- dcode: integer (nullable = true)
 |-- dname: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- lng: double (nullable = true)

+---------+--------------------+--------------------+-----------+--------------------+-----+-------+-----------+-----------+
|id_depart|                name|             address|        tel|                time|dcode|  dname|        lat|        lng|
+---------+--------------------+--------------------+-----------+--------------------+-----+-------+-----------+-----------+
|        2|บิ๊กซี สาขาสุขาภิ...|643 ถนน รามคำแหง ...|0 2735 3062|เปิดเวลา 09:00-21...| 1006|บางกะปิ| 13.7708549|100.6583974|
|        3|  โลตัส จรัญสนิทวงศ์|244 ซอย จรัญสนิทว...|0 2434 7575|เปิดเวลา 08:00-22...| 1025|บางพลัด|13.78792845|100.5023849|
|        4|     โลตัส แจ้งวัฒนะ|300 

In [9]:
# Community
url_community = "https://cpudgiapp.bangkok.go.th/arcgis/rest/services/Community/Service_Community_Public/MapServer/0/query"
params = {
    "where": "1=1",
    "outFields": "*",
    "f": "json",
    "returnGeometry": "true"
}
headers = {"User-Agent": "Mozilla/5.0"}

response = requests.get(url_community, headers=headers, params=params)
response.raise_for_status()
data = response.json()

features = [f["attributes"] for f in data["features"]]

with tempfile.NamedTemporaryFile(mode="w", delete=False, suffix=".json") as tmp:
    for feature in features:
        tmp.write(json.dumps(feature) + "\n")
    temp_path = tmp.name

df_community = spark.read.json(temp_path)
df_community.printSchema()
df_community.show(5)


root
 |-- ADDRESS: string (nullable = true)
 |-- CMT_ID: double (nullable = true)
 |-- CMT_NAME: string (nullable = true)
 |-- CMT_TYPE: string (nullable = true)
 |-- COMMU_ADMIN_COMMU_BND_FINAL_CM: double (nullable = true)
 |-- COMMU_ADMIN_COMMU_BND_FINAL_DC: string (nullable = true)
 |-- COMMU_ADMIN_COMMU_BND_FINAL__1: string (nullable = true)
 |-- CREATED_DATE: string (nullable = true)
 |-- CREATED_USER: string (nullable = true)
 |-- DNAME: string (nullable = true)
 |-- EAST_BND: string (nullable = true)
 |-- ESTABLISHED: long (nullable = true)
 |-- FEMALE: double (nullable = true)
 |-- HOUSEHOLD: double (nullable = true)
 |-- LAST_EDITED_DATE: long (nullable = true)
 |-- LAST_EDITED_USER: string (nullable = true)
 |-- LAT: double (nullable = true)
 |-- LON: double (nullable = true)
 |-- MALE: double (nullable = true)
 |-- NGAN: double (nullable = true)
 |-- NHOUSE: double (nullable = true)
 |-- NORTH_BND: string (nullable = true)
 |-- OBJECTID: long (nullable = true)
 |-- ORIG_FID:

In [10]:
# School
url_school = "https://bmagis.bangkok.go.th/arcgis/rest/services/riskbkk/RISK_ADMIN_bma_school/FeatureServer/0/query"

params = {
    "where": "1=1",
    "outFields": "*",
    "f": "json",
    "returnGeometry": "true"
}
headers = {"User-Agent": "Mozilla/5.0"}

response = requests.get(url_school, headers=headers, params=params)
response.raise_for_status()
data = response.json()

features = [f["attributes"] for f in data["features"]]

with tempfile.NamedTemporaryFile(mode="w", delete=False, suffix=".json") as tmp:
    for feature in features:
        tmp.write(json.dumps(feature) + "\n")
    temp_path = tmp.name

df_school= spark.read.json(temp_path)
df_school.printSchema()
df_school.show(5)


root
 |-- ADDRESS: string (nullable = true)
 |-- CREATED_DATE: long (nullable = true)
 |-- CREATED_USER: string (nullable = true)
 |-- DCODE: string (nullable = true)
 |-- GLOBALID: string (nullable = true)
 |-- LAST_EDITED_DATE: long (nullable = true)
 |-- LAST_EDITED_USER: string (nullable = true)
 |-- NAME: string (nullable = true)
 |-- OBJECTID: long (nullable = true)
 |-- TYPE: string (nullable = true)
 |-- X: double (nullable = true)
 |-- Y: double (nullable = true)

+--------------------+------------+------------+-----+--------------------+----------------+----------------+--------------------+--------+----+----------+-----------+
|             ADDRESS|CREATED_DATE|CREATED_USER|DCODE|            GLOBALID|LAST_EDITED_DATE|LAST_EDITED_USER|                NAME|OBJECTID|TYPE|         X|          Y|
+--------------------+------------+------------+-----+--------------------+----------------+----------------+--------------------+--------+----+----------+-----------+
|132 ถ.สามเสน แขวง

In [11]:
# Hospital
url_hospital = "https://bmagis.bangkok.go.th/arcgis/rest/services/riskbkk/RISK_ADMIN_Hospital/FeatureServer/0/query"

params = {
    "where": "1=1",
    "outFields": "*",
    "f": "json",
    "returnGeometry": "true"
}
headers = {"User-Agent": "Mozilla/5.0"}

# Get JSON
response = requests.get(url_hospital, headers=headers, params=params)
response.raise_for_status()
data_hospital = response.json()

# Extract attributes
features = [f["attributes"] for f in data_hospital["features"]]

with tempfile.NamedTemporaryFile(mode="w", delete=False, suffix=".json") as tmp:
    for feature in features:
        tmp.write(json.dumps(feature) + "\n")
    temp_path = tmp.name

# Spark
df_hospital = spark.read.json(temp_path)
df_hospital.printSchema()
df_hospital.show(5)


root
 |-- ADDRESS: string (nullable = true)
 |-- DCODE: string (nullable = true)
 |-- NAME: string (nullable = true)
 |-- NUM_BED: long (nullable = true)
 |-- OBJECTID: long (nullable = true)
 |-- TEL: string (nullable = true)
 |-- X: double (nullable = true)
 |-- Y: double (nullable = true)

+--------------------+-----+--------------------+-------+--------+------------------+----------+-----------+
|             ADDRESS|DCODE|                NAME|NUM_BED|OBJECTID|               TEL|         X|          Y|
+--------------------+-----+--------------------+-------+--------+------------------+----------+-----------+
|171 ถ.พหลโยธิน แข...| 1042|โรงพยาบาลภูมิพลอด...|    700|       1|     0 - 2534-7000|674848.512|1538263.349|
|312 ถ. ราชวิถี แข...| 1037|โรงพยาบาลและสถาบั...|    350|       2|   0 - 2246 - 9254|665118.241|1522693.716|
|315 ถ.ราชวิถี ทุ่...| 1037|โรงพยาบาลพระมงกุฏ...|   1236|       3|0 - 2246 - 1400-28|665789.636|1522522.013|
|2 ถ.พญาไท แขวงทุ่...| 1037|    โรงพยาบาลราชวิถี|   

In [12]:
def clean_and_convert_to_gdf(df, col_map, place_type, drop_zero=True):
    # Step 1: select & rename
    df_clean = df.select(
        F.col(col_map["name"]).alias("name"),
        F.col(col_map["lat"]).alias("lat"),
        F.col(col_map["lng"]).alias("lng")
    )

    # Step 2: clean name
    df_clean = df_clean.withColumn("name", F.trim(F.col("name").cast(StringType())))
    df_clean = df_clean.filter((F.col("name").isNotNull()) & (F.col("name") != ""))

    # Step 3: convert numeric
    df_clean = df_clean.withColumn("lat", F.col("lat").cast(DoubleType()))
    df_clean = df_clean.withColumn("lng", F.col("lng").cast(DoubleType()))

    # Step 4: drop missing / zero
    df_clean = df_clean.filter(F.col("lat").isNotNull() & F.col("lng").isNotNull())
    if drop_zero:
        df_clean = df_clean.filter((F.col("lat") != 0) & (F.col("lng") != 0))

    # Step 5: detect coordinate system
    max_lat = df_clean.agg(F.max("lat")).collect()[0][0]

    if max_lat > 1000:
        # เป็น UTM → ต้อง convert เป็น WGS84
        print(f"[{place_type}] Detected UTM coordinates → converting to WGS84...")

        transformer = Transformer.from_crs("EPSG:32647", "EPSG:4326", always_xy=True)

        def utm_to_wgs84(lng, lat):
            x, y = transformer.transform(lng, lat)
            return float(y), float(x)

        schema = StructType([
            StructField("lat", DoubleType(), True),
            StructField("lng", DoubleType(), True)
        ])
        convert_udf = udf(utm_to_wgs84, schema)
        df_clean = df_clean.withColumn("coords", convert_udf(F.col("lng"), F.col("lat")))
        df_clean = df_clean.withColumn("lat", F.col("coords.lat")).withColumn("lng", F.col("coords.lng"))
        df_clean = df_clean.drop("coords")

    # Step 6: create geometry column (WKT)
    df_clean = df_clean.withColumn("geometry", F.concat(F.lit("POINT("), F.col("lng"), F.lit(" "), F.col("lat"), F.lit(")")))

    # Step 7: add type column
    df_clean = df_clean.withColumn("type", F.lit(place_type))

    return df_clean

In [13]:
column_mapping = {
    "department": {"name": "name", "lat": "lat", "lng": "lng"},
    "community":  {"name": "CMT_NAME", "lat": "LAT",  "lng": "LON"},
    "school":     {"name": "NAME",     "lat": "Y",    "lng": "X"},
    "hospital":   {"name": "NAME",     "lat": "Y",    "lng": "X"},
}


In [14]:
gdf_department = clean_and_convert_to_gdf(
    df_department, column_mapping["department"], "department"
)

gdf_community = clean_and_convert_to_gdf(
    df_community,  column_mapping["community"],  "community"
)

gdf_school = clean_and_convert_to_gdf(
    df_school,     column_mapping["school"],     "school"
)

gdf_hospital = clean_and_convert_to_gdf(
    df_hospital,   column_mapping["hospital"],   "hospital"
)

[school] Detected UTM coordinates → converting to WGS84...
[hospital] Detected UTM coordinates → converting to WGS84...


In [15]:
# รวมทุกประเภทเป็น DataFrame เดียว
gdf_public_place = gdf_department\
    .unionByName(gdf_community) \
    .unionByName(gdf_school) \
    .unionByName(gdf_hospital)

# นับจำนวนแถวทั้งหมด
print("Total public places:", gdf_public_place.count())

# แสดงตัวอย่าง 5 แถวแรก
gdf_public_place.show(5, truncate=False)

Total public places: 1718
+------------------------------------------+-----------+-----------+------------------------------+----------+
|name                                      |lat        |lng        |geometry                      |type      |
+------------------------------------------+-----------+-----------+------------------------------+----------+
|บิ๊กซี สาขาสุขาภิบาล 3 สาขา 2 (เอ็กซ์ตร้า)|13.7708549 |100.6583974|POINT(100.6583974 13.7708549) |department|
|โลตัส จรัญสนิทวงศ์                        |13.78792845|100.5023849|POINT(100.5023849 13.78792845)|department|
|โลตัส แจ้งวัฒนะ                           |13.89582877|100.5561529|POINT(100.5561529 13.89582877)|department|
|โลตัส ซีคอนสแควร์                         |13.69253958|100.6480293|POINT(100.6480293 13.69253958)|department|
|โลตัส บางแค                               |13.71265311|100.4189684|POINT(100.4189684 13.71265311)|department|
+------------------------------------------+-----------+-----------+------------------